In [4]:
import requests
import re
from selenium import webdriver
from bs4 import BeautifulSoup
import urllib.request
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import NoSuchElementException
import time
SCROLL_PAUSE_TIME = 1.2
from datetime import date
import pandas as pd
date_today = date.today()

In [94]:
name = []
product_url = []
sales_price = []
price = []
timestamp  = []
brand = []
product_type = []
image_url = []

#for i in range(0, 421, 12):
for i in range(0, 145, 12):
    #URL = 'https://www.carhartt.com/category/carhartt-men#facet:&productBeginIndex:{}&orderBy:&pageView:grid&minPrice:&maxPrice:&pageSize:&'.format(i)
    URL = 'https://www.carhartt.com/category/womens#facet:&productBeginIndex:{}&orderBy:&pageView:grid&minPrice:&maxPrice:&pageSize:&'.format(i)
    driver = webdriver.Chrome('/Users/mandili/.wdm/drivers/chromedriver/87.0.4280.20/mac64/chromedriver')
    driver.get(URL)
    soup = BeautifulSoup(driver.page_source,'html.parser')
    
    for i in soup.find_all('div',{'class':'small-6 large-4 columns product-cell'}):
        timestamp.append(date_today)
        brand.append('Carhartt')
        product_type.append(None) # one parameter for function
        name.append(i.find('h5',{'class':'t14'}).text)
        product_url.append(i.find('div',{'class':'product-description'}).a.get('href'))
        sales_price.append(i.find('p',{'class':'price t14 not-bold'}).span.text)
        price.append(None)
        image_url.append(str(i.div.div.div.a.img.get('data-lazy-src')).replace('400L2', '600L2'))

In [95]:
column_names = ["brand", "product_type", "product_name", "price", 
                "sales_price", "product_url", "picture_url", "timestamp"]
first = pd.DataFrame(columns = column_names)
first.brand = brand
first.product_type = product_type
first.product_name = name
#first.product_description = product_description
first.price = price
first.sales_price = sales_price
#first.color = color
#first.material = material
first.product_url = product_url
first.picture_url = image_url
first.timestamp = timestamp
first.head()

,brand,product_type,product_name,price,sales_price,product_url,picture_url,timestamp
0,Carhartt,None,Women's Washed Duck Insulated Active Jac,None,$109.99,https://www.carhartt.com/products/carhartt-wom...,https://imgcdn.carhartt.com/is/image/Carhartt/...,2020-12-04
1,Carhartt,None,Women's Washed Duck Sherpa Lined Jacket,None,$119.99,https://www.carhartt.com/products/carhartt-wom...,https://imgcdn.carhartt.com/is/image/Carhartt/...,2020-12-04
2,Carhartt,None,Carhartt® Washed Duck Sherpa Lined Vest,None,$79.99,https://www.carhartt.com/products/carhartt-wom...,https://imgcdn.carhartt.com/is/image/Carhartt/...,2020-12-04
3,Carhartt,None,Quilt-Lined Duck Bib Overall,None,$99.99,https://www.carhartt.com/products/carhartt-wom...,https://imgcdn.carhartt.com/is/image/Carhartt/...,2020-12-04
4,Carhartt,None,Carhartt® Yukon Extremes® Insulated Parka,None,$299.99,https://www.carhartt.com/products/carhartt-wom...,https://imgcdn.carhartt.com/is/image/Carhartt/...,2020-12-04


In [47]:
first.to_csv("first-carhartt.csv", index = False, encoding = 'utf-8-sig')

# Second layer: color, description, material

In [96]:
color = []
product_description = []
material = []
picture_name = []

counter = 1
for link in product_url:
    item_color = []
    ourUrl = urllib.request.urlopen(link)
    soup = BeautifulSoup(ourUrl,'html.parser')
    for color_i in soup.find_all('li',{'class':'pdpRedesign'}):
        item_color.append(color_i.find('a').get('title'))
    index_start = str(soup.find('div',{'id':'desc'})).find('</p>')
    end_index = str(soup.find('div',{'id':'desc'})).find('<b>')
    product_description.append(str(soup.find('div',{'id':'desc'}))[index_start+4:end_index].replace('<ul><li>', ' ').replace('</li><li>', ' ').replace('</li></ul>',' ').strip())
    material.append(None)
    color.append(item_color)
    picture_name.append(counter)
    counter += 1

In [97]:
column_names = ["brand", "product_type", "product_name", 'product_description', "price", 
                "sales_price", "color","material", "product_url", "picture_url", "timestamp"]
second_pass = pd.DataFrame(columns = column_names)
second_pass.brand = brand
second_pass.product_type = product_type
second_pass.product_name = name
second_pass.product_description = product_description
second_pass.price = price
second_pass.sales_price = sales_price
second_pass.color = color
second_pass.material = material
second_pass.product_url = product_url
second_pass.picture_url = image_url
second_pass.timestamp = timestamp
second_pass.head()

,brand,product_type,product_name,product_description,price,sales_price,color,material,product_url,picture_url,timestamp
0,Carhartt,None,Women's Washed Duck Insulated Active Jac,"An update to our most popular Active Jac, the ...",None,$109.99,"[Black, Carhartt Brown, Dark Brown, Magnesium,...",None,https://www.carhartt.com/products/carhartt-wom...,https://imgcdn.carhartt.com/is/image/Carhartt/...,2020-12-04
1,Carhartt,None,Women's Washed Duck Sherpa Lined Jacket,This women's hooded jacket is built to brave w...,None,$119.99,"[Black, Deep Wine, Taupe Gray, Navy, Bay Green]",None,https://www.carhartt.com/products/carhartt-wom...,https://imgcdn.carhartt.com/is/image/Carhartt/...,2020-12-04
2,Carhartt,None,Carhartt® Washed Duck Sherpa Lined Vest,This women's vest is a versatile layer built t...,None,$79.99,"[Dark Brown, Taupe Gray, Navy, Redwood, Carhar...",None,https://www.carhartt.com/products/carhartt-wom...,https://imgcdn.carhartt.com/is/image/Carhartt/...,2020-12-04
3,Carhartt,None,Quilt-Lined Duck Bib Overall,"Made to hold their own through cold and wind, ...",None,$99.99,"[Navy, Black, Dark Brown]",None,https://www.carhartt.com/products/carhartt-wom...,https://imgcdn.carhartt.com/is/image/Carhartt/...,2020-12-04
4,Carhartt,None,Carhartt® Yukon Extremes® Insulated Parka,Carhartt Yukon Extremes gear is engineered for...,None,$299.99,"[Burnt Olive, Black]",None,https://www.carhartt.com/products/carhartt-wom...,https://imgcdn.carhartt.com/is/image/Carhartt/...,2020-12-04


In [98]:
second_pass.to_csv('carhartt-women.csv', index = False, encoding='utf-8-sig')

# Download Images

In [101]:
import os
import requests as req

new_pd = pd.read_csv('/Users/mandili/Desktop/carhartt-men.csv', encoding='utf-8-sig')

os.mkdir('carhartt-men')
i = 1
for index, link in enumerate(new_pd['picture_url']):
    img_data = req.get(link).content
    with open('carhartt-men/'+ "carhartt-men_"+str(index+1)+'.jpg','wb+') as f:
        f.write(img_data)
    i+=1

In [102]:
import os
import requests as req

new_pd = pd.read_csv('/Users/mandili/Desktop/carhartt-women.csv', encoding='utf-8-sig')

os.mkdir('carhartt-women')
i = 1
for index, link in enumerate(new_pd['picture_url']):
    img_data = req.get(link).content
    with open('carhartt-women/'+ "carhartt-women_"+str(index+1)+'.jpg','wb+') as f:
        f.write(img_data)
    i+=1